# VGGNet

https://arxiv.org/pdf/1409.1556.pdf

본 코드는 아래의 실습을 참고했습니다.

https://drive.google.com/drive/folders/12zphz36T6gEJac6WScnvRN27-f1tfHO1

VGGNet은 conv 연산이 2번, 3번이 반복되는 블록으로 총 16개의 레이어로 구성된 아키텍쳐를 구성해보겠습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

In [2]:
def conv_2_block(in_dim, out_dim): # 2개의 합성곱 레이어를 만드는 콘브 블럭입니다.
    model = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.conv2d(out_dim, out_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
    )
    return model

def conv_3_block(in_dim, out_dim): # 3개의 합성곱 레이어를 만드는 콘브 블럭입니다.
    model = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.conv2d(out_dim, out_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.conv2d(out_dim, out_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
    )
    return model

In [3]:
class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3, base_dim),
            conv_2_block(base_dim, 2*base_dim),
            conv_3_block(2*base_dim, 4*base_dim),
            conv_3_block(4*base_dim, 8*base_dim),
            conv_3_block(8*base_dim, 8*base_dim)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 7 * 7, 100),
            nn.ReLU(True),
            nn.Linear(100, 20),
            nn.ReLU(True),
            nn.Linear(20, num_classes)
        )
    def forward(x):
        x = self.layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

다음은 파이토치 공식 구현 버전을 살펴보겠습니다.

https://github.com/pytorch/vision/tree/master/torchvision/models

In [4]:
class VGG(nn.Module):
    
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features # make_layers로 만든 features를 집어 넣는다.
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # avgpool이 feature와 classifier 사이에 들어갑니다.
        # Adaptive Pooling은 그냥 Pooling과는 다르게 S, K, P 를 지정하지 않고 인풋과 아웃풋에 맞춰서 자동으로 셋팅 됩니다.
        self.classifier = nn.Sequential( # 마지막 classifier 가 들어갑니다.
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes)
        )
        if init_weights: # init_weights 인수가 True면
            self._initialize_weights() # 이니셜라이즈를 불러옵니다.
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d): # isinstance(x, y) 는 x가 y와 같은지 확인하는 함수다. 같으면 True를 반환한다.
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def make_layers(cfg, batch_norm=False): # cfg 를 인수로 받습니다. cfg는 모델 아키텍쳐입니다. 아래 참고.
        layers = [] # 레이어를 리스트로 먼저 할당.
        in_channels = 3 # 인 채널은 3으로 셋팅
        for v in cfg: # cfg에서 엘리먼트를 하나씩 뺍니다.
            if v == 'M': # cfg의 엘리먼트가 M, 즉 맥스풀링이면 레이어에 맥스풀링을 더합니다.
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else: # 아니라면 합성곱 레이어를 합칩니다.
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm: # 여기서 배치노멀라이제이션이 존재하면 사이에 껴주고
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else: # 아니라면 그냥 컨브에 렐루만 넣어줍니다.
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v # in_channels는 나가는 채널의 수로 바꿔줍니다.
        return nn.Sequential(*layers) # 시퀀셜로 묶어서 내보내줍니다.

In [5]:
cfgs = {
        'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
        'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
        }